Основано на блокноте https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb

Новый набор данных получен благодаря аугментации из ноутбука data_augmentation

In [ ]:
!pip install datasets transformers seqeval razdel

     |████████████████████████████████| 290 kB 5.3 MB/s 
     |████████████████████████████████| 3.1 MB 36.9 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 41.9 MB/s 
     |████████████████████████████████| 59 kB 5.8 MB/s 
     |████████████████████████████████| 132 kB 40.4 MB/s 
     |████████████████████████████████| 243 kB 44.9 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
     |████████████████████████████████| 596 kB 44.3 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 192 kB 47.3 MB/s 
     |████████████████████████████████| 160 kB 41.7 MB/s 
     |████████████████████████████████| 271 kB 45.9 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=88bd6414c53a1adeff4e4f36758c0b3f8c9b959c301b627be3883e8efa21ec9e
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72

In [ ]:
import json
import numpy as np
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
import random
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import log_loss

In [ ]:
model_checkpoint = "cointegrated/LaBSE-en-ru"
batch_size = 8


In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
import re
import pandas as pd
df = pd.read_csv('new_train.csv', sep=';')
df.text_to_span = df.text_to_span.apply(lambda x: ' '.join(x.split()))
df.text_to_span = df.text_to_span.apply(lambda x: x.replace("'",'').replace('’','').replace('\n','').replace('‘','').lower().split(','))
df.text_to_span.values[0]

['раздавила себе сосок', ' постоянно бежит молоко']

In [ ]:
test = pd.read_csv('test.csv')


приводим данные в нужный вид:

In [ ]:
with open ('tag_data.json','r') as f:
  tag_data = json.load(f)

In [ ]:
data = []
sentence_ids = []
for item in tag_data:

  if len(df[df['text'] == item['data']['text']]) > 0 and len(item['annotations'][0]['result']) != 0:
    sentence_ids.append(df[df['text'] == item['data']['text']].index[0])
    data.append({'text':item['data']['text'],'sentence_id':df[df['text'] == item['data']['text']].index[0],'entities':[]})
    count = -1
    for tag in item['annotations'][0]['result']:
      count += 1
      data_ent = {'entity_id':count,'entity_text':tag['value']['text'].lower(),'entity_type':'B',
                                        'start':tag['value']['start'],'end':tag['value']['end']}
      data[-1]['entities'].append(data_ent)
      

In [ ]:
bad_ids = []
for _,row in df.iterrows():
  if _ not in sentence_ids:
    bad_ids.append(_)

In [ ]:
df = df.drop(bad_ids)
df = df.drop('Unnamed: 0',axis = 1)

Напишем функцию, перекладывающую разметку сущностей на уровень слов. Будем использовать [IOB](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging))-нотацию, чтобы разделять несколько сущностей одного типа, идущих подряд. 

In [ ]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item['text']))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['text'])
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item['entities']:
        e_words = sorted({idx for idx in char2word[e['start']:e['end']] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e['entity_type']
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e['entity_type']

    return {'tokens': words, 'tags': word_labels}

In [ ]:
for item in data:
  res = extract_labels(item)
  if all(x not in res['tags'] for x in ['B-B', 'I-B']):
    print(res)

In [ ]:
from sklearn.model_selection import train_test_split
ner_data = []
for item in data:
  ner_data.append(extract_labels(item)) 
ner_train, ner_test = train_test_split(ner_data, test_size=0.05, random_state=1)

Пример данных

In [ ]:
len(data)

554

In [ ]:
ner_train[0]

{'tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'tokens': ['Здравствуйте',
  ',',
  'опять',
  'нужна',
  'помощь',
  ',',
  'у',
  'моей',
  'коровы',
  'под',
  'левым',
  'глазом',
  'появилась',
  'шишка',
  '.',
  'Вет',
  'не',
  'идет',
  'посмотреть',
  ',',
  'сказала',
  'наверное',
  'укусил',
  'кто',
  ',',
  'велела',
  'мазать',
  'левомеколью',
  '.',
  'Ничего',
  'не',
  'проходит',
  'даже',
  'больше',
  'стала',
  '.',
  'Что',
  'делать',
  '?',
  'Помогите',
  'подскажите',
  'пожалуйста',
  '.']}

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
201,"[Скажите, пожалуйста, ,, телёнку, месяц, и, двадцать, дней, ,, даю, молоко, 2, л, 3, раза, ,, кушает, сено, ,, но, при, введении, молотого, ячмени, и, овса, стул, стал, жидким, ,, сначала, белым, ,, теперь, темным, ., Это, нормально, или, нет, ,, таблетки, от, поноса, давали, не, помогают, ., Те...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-B, I-B, I-B, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-B, O, O, O, O, O, O, O, O, O, O, O, O, O]"
446,"[Телёнок, сильно, поносит, ,, чем, помогает, ,, скажите, .]","[O, O, B-B, O, O, O, O, O, O]"
237,"[У, теленка, (, 2, месяца, ), утром, заложили, нос, и, отказались, от, молока, ., Что, делать, ,, скажите, пожалуйста, ?]","[O, O, O, O, O, O, O, B-B, I-B, O, B-B, I-B, I-B, O, O, O, O, O, O, O]"


Соберём все виды меток в список. 

In [ ]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O', 'B-B', 'I-B']

Сложим наши данные в объект [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), нативный для huggingface.

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 526
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 28
    })
})

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/806 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/509k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [2, 6467, 16, 735, 545, 923, 20221, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],is_split_into_words=True)

{'input_ids': [2, 6467, 16, 735, 545, 923, 20221, 11447, 1190, 4596, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [ ]:
example = ner_train[5]
print(example["tokens"])

['Что', 'делать', 'если', 'корова', 'запускает', 'одну', 'долю', 'вымени', 'и', 'горячую', '?', 'Здравствуйте', ',', 'подскажите', ',', 'корова', 'в', 'запуске', 'опухла', 'одна', 'доля', ',', 'остальные', 'мягкие', ',', 'телится', 'через', 'месяц', '.', 'Сталкиваюсь', 'с', 'этим', 'впервые', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Что', 'делать', 'если', 'кор', '##ова', 'запуска', '##ет', 'одну', 'долю', 'вы', '##мени', 'и', 'горячую', '?', 'Здравствуйте', ',', 'под', '##скажите', ',', 'кор', '##ова', 'в', 'запуск', '##е', 'оп', '##ух', '##ла', 'одна', 'доля', ',', 'остальные', 'мягкие', ',', 'тели', '##тся', 'через', 'месяц', '.', 'Ст', '##ал', '##ки', '##ваю', '##сь', 'с', 'этим', 'впервые', '.', '[SEP]']


Чтобы перейти с уровня слов на уровень subword tokens, нужно ещё раз предобработать тексты.

In [ ]:
len(example["tags"]), len(tokenized_input["input_ids"])

(34, 49)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 3, 4, 4, 5, 6, 7, 7, 8, 9, 10, 11, 12, 13, 13, 14, 15, 15, 16, 17, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 25, 26, 27, 28, 29, 29, 29, 29, 29, 30, 31, 32, 33, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

49 49


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the flag `label_all_tokens`.

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True,max_length=512)
    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 14086, 5, 48902, 10585, 5, 12358, 3278, 3767, 38145, 4779, 41824, 695, 16, 44259, 8772, 26880, 12738, 39757, 4813, 34972, 940, 16, 346, 662, 29269, 355, 6639, 3841, 1256, 18, 27463, 340, 1730, 789, 6292, 357, 8919, 36375, 650, 30377, 654, 12390, 51617, 695, 16, 527, 51691, 27938, 12366, 40991, 16, 575, 36731, 41774, 16, 357, 8919, 16172, 17263, 48918, 18, 1910, 2556, 11636, 41958, 983, 35, 8254, 1077, 8471, 12512, 1330, 35, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
label_list

['O', 'B-B', 'I-B']

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Downloading:   0%|          | 0.00/492M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized f

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")


Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [ ]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'B': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:

import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Модель недообучилась: похоже, что нужно обучить больше слоёв. Разморозим их все (но, воможно, более правильно было бы разморозить лишь несколько верхних), и поучимся ещё эпох 20.

In [ ]:
#разморозка
for param in model.parameters():
    param.requires_grad = True

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
#~30 loss best
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags.
***** Running training *****
  Num examples = 526
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 396


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.251691,0.406061,0.507576,0.451178,0.904195
2,No log,0.215394,0.484848,0.484848,0.484848,0.916100
3,No log,0.187369,0.578571,0.613636,0.595588,0.930839
4,No log,0.186516,0.585185,0.598485,0.591760,0.933107
5,No log,0.193922,0.632812,0.613636,0.623077,0.931406


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags.
***** Running Evaluation *****
  Num examples = 28
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags.
***** Running Evaluation *****
  Num examples = 28
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags.
***** Running Evaluation *****
  Num examples = 28
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags.
***** Running Evaluation *****
  Num examples = 28
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForToke

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.251691,0.406061,0.507576,0.451178,0.904195
2,No log,0.215394,0.484848,0.484848,0.484848,0.916100
3,No log,0.187369,0.578571,0.613636,0.595588,0.930839
4,No log,0.186516,0.585185,0.598485,0.591760,0.933107
5,No log,0.193922,0.632812,0.613636,0.623077,0.931406
6,No log,0.196079,0.627907,0.613636,0.620690,0.933673


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags.
***** Running Evaluation *****
  Num examples = 28
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=396, training_loss=0.18481676506273675, metrics={'train_runtime': 234.311, 'train_samples_per_second': 13.469, 'train_steps_per_second': 1.69, 'total_flos': 300490924870548.0, 'train_loss': 0.18481676506273675, 'epoch': 6.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags.
***** Running Evaluation *****
  Num examples = 28
  Batch size = 8


{'epoch': 6.0,
 'eval_accuracy': 0.9336734693877551,
 'eval_f1': 0.6206896551724139,
 'eval_loss': 0.19607938826084137,
 'eval_precision': 0.627906976744186,
 'eval_recall': 0.6136363636363636,
 'eval_runtime': 0.6647,
 'eval_samples_per_second': 42.122,
 'eval_steps_per_second': 6.017}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags.
***** Running Prediction *****
  Num examples = 28
  Batch size = 8


{'B': {'f1': 0.6206896551724139,
  'number': 132,
  'precision': 0.627906976744186,
  'recall': 0.6136363636363636},
 'overall_accuracy': 0.9336734693877551,
 'overall_f1': 0.6206896551724139,
 'overall_precision': 0.627906976744186,
 'overall_recall': 0.6136363636363636}

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-B,I-B
O,1439,15,24
B-B,41,87,4
I-B,14,19,121


# Применение модели

Более простое применение модели: пайплайн от huggingface

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [ ]:
print(test.iloc[0].text)
print(pipe(test.iloc[0].text))

Понос у месячных телят. Подскажите методы и способы лечения жидкого стула у месячных телят. Хватают в рот и лижут всё что видят, сено, солому, лижут соль. Потом страдают. И мы вместе с ними.
[{'entity_group': 'B', 'score': 0.99579334, 'word': 'Понос', 'start': 0, 'end': 5}, {'entity_group': 'B', 'score': 0.9717525, 'word': 'жидкого стула', 'start': 60, 'end': 73}, {'entity_group': 'B', 'score': 0.662967, 'word': 'лижут', 'start': 143, 'end': 148}]


## Classification

### Data preparation

In [ ]:
texts = []

for _,row in df.iterrows():
  data = pipe(row.text)
  if len(data) > 0:
    data = ' '.join([x['word'] for x in data])
  else:
    print(row['text'])
    data = row['text']
  texts.append(data)
df_new = df.drop('text_to_span',axis=1)
df_new['text'] = texts
df_new.head()

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


У меня первотелка на днях, у нее левый передний ящик как будто пуст, остальные три бухли и дольки тоже, скажите что делать, а до отела не стоит волноваться?


,text_id,text,эймериоз,абсцесс,диспепсия молодняка,остертагиоз,мастит,инфекционный ринотрахеит,отёк вымени,тенденит,сибирская язва,лишай,другое
0,0,раздавила себе сосок в хлам соска постоянно бежит молоко,0,0,0,0,1,0,0,0,0,0,0
1,1,появились бугорки корка гнойные язвы,0,0,0,0,0,0,0,0,1,1,1
2,2,Молоко течёт слабые соски молоко бежит Молоко течёт молоко порой само бежит соски слабые,0,0,0,0,1,0,1,0,0,0,0
3,3,болячки на вымени вымя покрылось маленькими болячками коричневые точки,0,0,0,0,0,0,0,0,1,1,1
4,4,сосок как будто набухли,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
df_new = df_new.drop('text_id',axis=1)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(pd.DataFrame(df_new['text']), df_new[df_new.columns[1:-1]], test_size=0.4,random_state = 1,shuffle = True)

In [ ]:
train_encodings = tokenizer(list(X_train.text.values), truncation=True, padding=True, max_length=512)
valid_encodings = tokenizer(list(X_val.text.values), truncation=True, padding=True, max_length=512)


In [ ]:
class CowDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = CowDataset(train_encodings, y_train.values)
valid_dataset = CowDataset(valid_encodings, y_val.values)

### Model training

In [ ]:
from sklearn.metrics import log_loss

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions
  # calculate accuracy using sklearn's function
  loss = log_loss(labels, preds)/len(labels)
  return {
      'log_loss': loss,
  }

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model_cls = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = y_train.shape[1], problem_type="multi_label_classification")

loading configuration file https://huggingface.co/cointegrated/LaBSE-en-ru/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/b36ab435ca53051f0bff3b687c259e0e48d9659dccc000e42ff965e25bdc88a4.8a79cb009e1f2ff400bf480c899cc74bd1da7cf94bd051ff4854de3b447b00f2
Model config BertConfig {
  "_name_or_path": "cointegrated/LaBSE-en-ru",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
  

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=8,              # total number of training epochs
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=batch_size,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.005,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    #load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    learning_rate=3e-5,
    evaluation_strategy="epoch",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model_cls,                     # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 165
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 42


Epoch,Training Loss,Validation Loss,Loss,Runtime,Samples Per Second,Steps Per Second
1,No log,0.023048,0.209272,3.461000,111.528000,14.158000
2,No log,0.021035,0.207284,3.460500,111.544000,14.160000


***** Running Evaluation *****
  Num examples = 386
  Batch size = 8
***** Running Evaluation *****
  Num examples = 386
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=42, training_loss=0.05485312711624872, metrics={'train_runtime': 17.0239, 'train_samples_per_second': 19.384, 'train_steps_per_second': 2.467, 'total_flos': 7970987482920.0, 'train_loss': 0.05485312711624872, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 386
  Batch size = 8


{'epoch': 2.0,
 'eval_log_loss': 0.021035367458354288,
 'eval_loss': 0.20728442072868347,
 'eval_runtime': 3.6993,
 'eval_samples_per_second': 104.345,
 'eval_steps_per_second': 13.246}

# Submission

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model_cls(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs

In [ ]:
hardcode_ner_data = []
for _,row in df.iterrows():
  for span in row['text_to_span']:
    for word in span.split():
      if len(word) > 3:
        hardcode_ner_data.append(word)
hardcode_ner_data[:5]

['раздавила', 'себе', 'сосок', 'постоянно', 'бежит']

In [ ]:
def extract_hardcode(text):
  words = text.split()
  true_words = []
  for word in words:
    if word in hardcode_ner_data:
      true_words.append(word)
  if len(true_words) == 0:
    return text
  spans = []
  for word in true_words:
    for match in re.finditer(word,text):
      spans.append([match.start(),match.end()])
  print(spans)
  return ' '.join(true_words),spans


test = pd.read_csv('test.csv')
texts = []
spans = []
for _,row in test.iterrows():
  data = pipe(row.text.lower())
  spans_temp = []
  for sp in data:
    spans_temp.append([sp['start'],sp['end']])
  if len(data) > 0:
    data = ' '.join([x['word'] for x in data])
  else:
    data,span = extract_hardcode(row['text'].lower())
    for sp in span:
      spans_temp.append(sp)
    print(data)
  spans.append(spans_temp)
  texts.append(data)
test['text'] = texts
test.head()


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[[28, 39]]
температура
[[23, 32], [33, 39], [91, 97]]
захромала корова
[[48, 54], [101, 105], [146, 152], [180, 185], [224, 229], [232, 237], [355, 360]]
корова было совсем около лежит очень может


,text_id,text
0,294,понос жидкого стула лижут
1,295,понос понос понос понос с кровью жидкий стул с кровью поносить плохо встаёт и ест
2,296,отнимаются ноги ноги отнимаются не поносит встать не может
3,297,недомогание вялость
4,298,сгустки у сгустки крови у в молоке


In [ ]:
preds = list(map(get_prediction,test['text']))

In [ ]:
clean_preds = []
for item in preds:
  temp_preds = []
  for p in item[0][:10]:
    temp_preds.append(p.item())
  clean_preds.append(temp_preds)

In [ ]:

preds_df = pd.DataFrame(clean_preds)

In [ ]:
submission_columns = ['text_id'] + list(df.drop('text_to_span',axis=1).columns[2:-1])
submission = pd.concat([test['text_id'], preds_df], axis=1)
submission.columns = submission_columns

In [ ]:
submission['span'] = spans

In [ ]:
submission_json = {str(k): {"span": v['span'], "label": list({i:v[i] for i in v if i!='span'}.values())} \
                   for k,v in submission.set_index('text_id').to_dict('index').items()}

with open('submission_v73_ner_clsbert.json', 'w') as final_submit:
    json.dump(submission_json, final_submit, indent=4)